# imports

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from schlichtanders.mymatplotlib import Centre
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import os, platform, sys
from pprint import pprint

In [3]:
__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [4]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session, create_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
import operator as op
from collections import defaultdict
import csv
import heapq
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
inf = float('inf')

In [5]:
import experiment_toy_models
import experiment_util
from schlichtanders.mycontextmanagers import ignored
from schlichtanders.myobjects import Namespace
from evaluation import get_modes
import evaluation as eva

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [6]:
import numpy as np
import theano

In [7]:
model_names = { # sorted by optimization type
    "ml": ['baselinedet'],
    "annealing": ['baseline', 'mixture', 'planarflow', 'planarflowdet', 'radialflow', 'radialflowdet'],
}
model_prefixes = reduce(op.add, model_names.values())

In [8]:
Hyper = experiment_util.get_toy_hyper()

# Collect models and find best ones

In [9]:
def key(fn, path):
    return "1d" in fn
best_hyper = eva.get_best_hyper(["toywithtestval", "toy_linux"], Hyper, model_prefixes, test_suffix=["best_val_loss"], key=key)

# Compute Results on Test-set

In [10]:
from sklearn import cross_validation

In [11]:
dim=1
error_func = experiment_util.RMSE
sampler = experiment_toy_models.toy_likelihood(dim=dim).function()

def data_gen(hyper):
    hyper.dim = dim
    x_true = np.array([hyper.x_true]*hyper.dim, dtype=theano.config.floatX)
    _Z = np.array([sampler(x_true) for n in range(1000)], dtype=theano.config.floatX)
    Z, TZ = cross_validation.train_test_split(_Z, test_size=0.1)  # 10% test used in paper
    Z, VZ = cross_validation.train_test_split(Z, test_size=0.1)  # 20% validation used in paper
    data = None, Z, None, VZ, None, TZ  # None represents X data
    return data, error_func

In [12]:
reload(eva)

<module 'evaluation' from '/home/stephan/GitProjects/theano_models/experiment/__file__'>

In [13]:
test_results = eva.compute_test_results(best_hyper, data_gen, model_module_id="toy", n_trials=5)

mixture
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
batch_precompile
batch_precompile

/home/stephan/GitProjects/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.rmsprop.RmsProp'>
  % (i, klass))
/home/stephan/GitProjects/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.adam.Adam'>
  % (i, klass))



1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 3

In [ ]:
%debug

In [ ]:
test_results

# Sample the approximate posterior distribution for evaluation

In [ ]:
best_hyper_samples = eva.sample_best_hyper(best_hyper, test_results)

# best_hyper_samples only refers to hypers which distribution can be compared (i.e. baselinedet and such is removed)
test_attrs = sorted(best_hyper_samples.keys())
model_prefixes = sorted(best_hyper_samples[test_attrs[0]].keys())
n_normflows = sorted(best_hyper_samples[test_attrs[0]][model_prefixes[0]].keys())

lp = len(model_prefixes)
lnn = len(n_normflows)

## Histogram (only possible for toy dataset)

In [ ]:
for i in range(n_best):
    fig, axs = plt.subplots(nrows=lp, ncols=len(n_normflows), sharex=True, sharey=True)

    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            if name == "baselinedet":
                continue
            samples = best_hyper_samples['best_val_loss'][name][nn][i]
            ax = next(axs_iter)
            ax.hist(samples, bins="auto") #50
            ax.tick_params(axis='both', which='major', labelsize=5)
            ax.tick_params(axis='both', which='minor', labelsize=5)
    #         ax.set_ylabel("frequency")
    #         ax.set_xlabel("parameter")
            ax.set_title("%s, %i" % (name, nn), fontsize=9)
    plt.xlim([-0.6, 1.1])
    fig.tight_layout()

## number of modes / histogram of modes

In [ ]:
best_modes = eva.get_best_modes(best_hyper_samples)
best_modes

In [ ]:
nr_modes = eva.get_nr_modes(best_hyper_samples)
nr_modes

## KL divergence

This must be computed on histograms, as we need an approximation for the densities/pms

In [ ]:
best_kl = eva.compute_kl(best_hyper_samples)

In [ ]:
def add_colorbar(ax, h):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="10%", pad=0.15)
    cbar = plt.colorbar(h, cax=cax)

In [ ]:
for nn in n_normflows:
    means = np.mean(best_kl['best_val_loss'][nn], axis=0)
    variances = np.var(best_kl['best_val_loss'][nn], axis=0)

    fig, axs = plt.subplots(ncols=2)
    h = axs[0].imshow(means, interpolation="None", cmap=plt.cm.gray_r) #norm=Centre(0)
    add_colorbar(axs[0], h)
#     plt.colorbar(h)

    h = axs[1].imshow(variances, interpolation="None", cmap=plt.cm.gray_r, norm=LogNorm())
    add_colorbar(axs[1], h)
#     plt.colorbar(h)
    plt.title("nn = %i" %nn)
    for ax in axs:
        ax.grid(False)

In [ ]:
variances